In [19]:
import pandas as pd
import numpy as np
import os
import time
import cv2
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import random

In [20]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Dense
from keras.layers import Flatten, Activation, Concatenate
from keras.losses import categorical_crossentropy
from keras.optimizers import adam

In [21]:
FOLDER = '../../data_kaggle/kuzushiji/'
IMAGES = FOLDER + 'train_images/'
CHARS = FOLDER + 'chars/'
MODELS = FOLDER + 'models/'
print(os.listdir(FOLDER))

['unicode_translation.csv', 'train_images', 'sample_submission.csv', 'train.csv', 'kuzushiji-recognition.zip', 'output', 'test_images', 'chars', 'models']


In [22]:
img_size = (1000, 1000, 1)
inputs = Input(shape=img_size, name='ssd_input')

## Block 1
conv1_1 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv1_1')(inputs)
conv1_2 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv1_2')(conv1_1)
pool1 = MaxPooling2D((2,2),strides=(2,2),padding='same',name='pool1')(conv1_2)

conv2_1 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv2_1')(pool1)
conv2_2 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv2_2')(conv2_1)
pool2 = MaxPooling2D((2,2),strides=(2,2),padding='same',name='pool2')(conv2_2)

conv3_1 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv3_1')(pool2)
conv3_2 = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv3_2')(conv3_1)
pool3 = MaxPooling2D((2,2),strides=(2,2),padding='same',name='pool3')(conv3_2)
flat3 = Flatten(name='flat3')(pool3)

dense1 = Dense(4787,activation='relu', name='ssd_dense1')(flat3)
dense2 = Dense(4787,activation='relu', name='ssd_dense2')(dense1)
dense3 = Dense(4787, activation='softmax', name='ssd_dense3')(dense2)
model =  Model(inputs, dense3)

In [23]:
from keras.initializers import Initializer

In [24]:
Initializer()

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ssd_input (InputLayer)       (None, 1000, 1000, 1)     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 1000, 1000, 64)    640       
_________________________________________________________________
conv1_2 (Conv2D)             (None, 1000, 1000, 64)    36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 500, 500, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 500, 500, 64)      36928     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 500, 500, 64)      36928     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 250, 250, 64)      0         
__________

In [26]:
model.compile(optimizer=adam(),
              loss=categorical_crossentropy)

In [27]:
model.layers

In [32]:
from keras.backend import get_session

In [34]:
session = get_session()

In [35]:
model.layers[1].kernel.initializer.run(session=session)

In [39]:
model.load_weights(MODELS + 'model.hdf5', by_name=True)

In [41]:
model.layers[2].get_weights()

[array([[[[-8.28759000e-02, -9.77700502e-02, -3.76123786e-02, ...,
           -6.30727559e-02,  2.07818374e-02, -4.77702469e-02],
          [ 9.12050232e-02, -3.25472765e-02, -8.75245966e-03, ...,
           -7.27493912e-02, -6.82048649e-02,  6.60538599e-02],
          [-5.67761511e-02, -2.85858810e-02,  3.98406573e-02, ...,
            6.28982903e-03, -4.31345627e-02, -4.03757542e-02],
          ...,
          [-1.28395572e-01, -3.28922272e-02, -9.22057703e-02, ...,
            5.09209484e-02, -4.17970121e-02, -1.59764234e-02],
          [ 7.18389377e-02, -5.57662658e-02, -7.28996247e-02, ...,
            3.28266397e-02,  5.24345264e-02,  2.36761589e-02],
          [-2.06681713e-02,  2.32366156e-02,  3.67710963e-02, ...,
            6.55118078e-02, -1.37306005e-01, -4.54532355e-03]],
 
         [[-7.94556960e-02, -6.43967837e-02,  5.66371642e-02, ...,
            1.14718908e-02, -6.21712655e-02,  1.07780034e-02],
          [ 6.75671697e-02,  4.31086719e-02,  8.55017975e-02, ...,
     